Model https://huggingface.co/mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization

Original colab https://colab.research.google.com/drive/1Ekd5pUeCX7VOrMx94_czTkwNtLN32Uyu?usp=sharing#scrollTo=oOoSrwWarJAC

In [32]:
display("hello")

'hello'

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Judging from the original colab this resolves to BertTokenizer 
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization")

# Judging from the original colab this resolves to EncoderDecoderModel
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization")

/opt/homebrew/Caskroom/miniconda/base/envs/NLP-summarization/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel
rouge = datasets.load_metric("rouge")

test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
r = 16
test_data = test_data.select(range(r))

batch_size = 16 # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
#     print(inputs)
    input_ids = inputs.input_ids#.to("cuda")
    attention_mask = inputs.attention_mask#.to("cuda")
    
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(output_str)
    
    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size)

pred_str = results["pred"]
label_str = results["highlights"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

Reusing dataset cnn_dailymail (/Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
100%|█████████████████████████████████████████████| 1/1 [00:09<00:00,  9.81s/ba]

['" the dukes of hazzard " died monday after a brief illness. his character became known for his distinctive " kew - kew " chuckle and for goofy catchphrases such as " cuff\'em! " upon making an arrest. " jimmy best was the most constantly creative person i have ever known, " co - star john schneider says.', 'randy zelin defends his client, dr. anthony moschetto, who faces criminal solicitation, conspiracy, burglary, arson, criminal prescription sale and weapons charges in connection to what prosecutors call a plot to take out a rival doctor on long island. he was released after posting $ 2 million bond and surrendering his passport.', 'president obama took part in a roundtable discussion this week on climate change and public health. he credits the clean air act with making americans " a lot " healthier, in addition to being able to " see the mountains in the background because they aren\'t covered in smog, " obama says.', 'the video features a gay couple holding hands, got 18 + ratin

In [33]:
# Showing the results
from IPython.core.display import display, HTML
from rouge import Rouge 

rouge_new = Rouge()
# scores = rouge.get_scores(hypothesis, reference)

for i in range(10):
    article = results["article"][i]
    highlight = results["highlights"][i]
    prediction = results["pred"][i]
    score = rouge_new.get_scores(highlight, prediction)
    
    rouge_1_f = score[0]["rouge-1"]["f"] * 100
    rouge_2_f = score[0]["rouge-2"]["f"] * 100
    s = f"rouge-1: {rouge_1_f}, rouge-2:  {rouge_2_f}" 
    
    match_article_prediction = display_string_matching(article, prediction, 
                                                       both=True, sentences=True,
                                                      titles=["Article", "Predicted Summary"])
    
    match_summary = display_string_matching(highlight, prediction, both=True, 
                                    sentences=False, 
                                    titles=["Real Summary", f"Predicted Summary ({s})"])
    
#     display(HTML(match_article_prediction)
    display(HTML(match_summary))
    print("---")

/var/folders/np/sw3cwd7x6rv0llrv26d2w1s40000gn/T/ipykernel_1875/592430296.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


---


---


---


---


...

---


---


---


---


---


In [31]:
# Credits tot Mauro Di Pietro
# https://towardsdatascience.com/text-summarization-with-nlp-textrank-vs-seq2seq-vs-bart-474943efeb09

import re
import difflib
import nltk
nltk.download()

'''
Find the matching substrings in 2 strings.
:parameter
    :param a: string - raw text
    :param b: string - raw text
:return
    2 lists used in to display matches
'''
def utils_split_sentences(a, b):
    ## find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks() if block.size > 20]
    
    ## difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)
    
    ## work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        ### a
        string = a[0 : first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a : m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size : next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size :]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        ### b
        string = b[0 : first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b : m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size : next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size :]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
    
    return lst_a, lst_b


'''
Highlights the matched strings in text.
:parameter
    :param a: string - raw text
    :param b: string - raw text
    :param both: bool - search a in b and, if True, viceversa
    :param sentences: bool - if False matches single words
:return
    text html, it can be visualized on notebook with display(HTML(text))
'''
def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()       
    
    ## highlight a
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)
    
    ## highlight b
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b) 
    second_text = ' '.join(second_text)
    
    ## concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text +'<br><br>'+ second_text
    return final_text

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
